In [ ]:
!pip install scikit-surprise

* pandas dan numpy: Untuk manipulasi data dan perhitungan numerik.
* cosine_similarity: Digunakan untuk menghitung kesamaan antara lagu-lagu dalam Content-Based Filtering.
* LabelEncoder: Untuk mengonversi data kategorikal (seperti genre dan artis) menjadi format numerik.
* SVD: Model dari surprise library untuk Collaborative Filtering.
* Reader dan Dataset: Untuk mempersiapkan data dalam format yang diterima oleh library surprise.
* train_test_split: Untuk membagi data menjadi data pelatihan dan pengujian.
* accuracy: Untuk mengevaluasi hasil prediksi dari model menggunakan metrik seperti MSE (Mean Squared Error).
* matplotlib: Untuk visualisasi data dan hasil model.

In [ ]:
#@title Import Libraries

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, Reader, Dataset
from surprise import accuracy
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.decomposition import TruncatedSVD
from joblib import Parallel, delayed
from sklearn.metrics import precision_score
from sklearn.preprocessing import MinMaxScaler



Fitur Dataset ada :
track_id, artists, album_name, track_name, popularity, duration_ms, explicit, danceability, energy, key, loudness, mode, speechiness, accousticness, instrumentalness, liveness, valence, tempo, time_signature, track_genre.

In [ ]:
#@title Load data

# Load the dataset
df = pd.read_csv('/content/dataset.csv')

# Cek beberapa data teratas untuk memastikan data terbaca dengan benar
print(df.head())
print(df.info())


   Unnamed: 0                track_id                 artists  \
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   


In [ ]:
#@title Data Preparation
# Cek Rows and Coloumn
nrows, ncols = df.shape
print(f'Dataset has {nrows} rows and {ncols} columns')

# Cek Duplikat Row dan Drop
duplicated_rows = df.duplicated().sum()
if duplicated_rows != 0:
    df = df.drop_duplicates()
    undroped_nrows, ncols = df.shape
    print(f'Total : {nrows - undroped_nrows} rows droped')

Dataset has 114000 rows and 21 columns


In [ ]:
#Cek Missing Values
total_rows_with_missing_values = (df.isnull().any(axis=1)).sum()
print(f'Total number Rows Missing: {total_rows_with_missing_values}')

Total number Rows Missing: 1


In [ ]:
#Drop Missing Value
index_to_drop = df[df.isnull().any(axis=1)].index
df.drop(index_to_drop, inplace=True)

print(f'Rows with missing values dropped. Updated DataFrame shape: {df.shape}')

Rows with missing values dropped. Updated DataFrame shape: (113999, 21)


In [ ]:
# Mengambil 23999 baris
df.drop(df.tail(90000).index, inplace=True)

# Menampilkan informasi ukuran DataFrame setelah penghapusan
print(f'Updated DataFrame shape after dropping rows: {df.shape}')

Updated DataFrame shape after dropping rows: (23999, 21)


* df.duplicated(): Menentukan baris yang duplikat berdasarkan seluruh kolom dalam dataset. Jika dua baris memiliki nilai yang sama di semua kolom, maka salah satunya dianggap duplikat.
* df.drop_duplicates(): Menghapus baris yang duplikat berdasarkan seluruh kolom atau berdasarkan kolom tertentu
* df.isnull(): Mengecek apakah ada nilai kosong (NaN) dalam dataset.
* df.drop(): Menghapus baris yang memiliki nilai kosong berdasarkan indeks yang ditemukan sebelumnya.

In [ ]:
print(df.tail)

<bound method NDFrame.tail of        Unnamed: 0                track_id                       artists  \
0               0  5SuOikwiRyPMVoIQDJUgSV                   Gen Hoshino   
1               1  4qPNDBW1i3p13qLCt0Ki3A                  Ben Woodward   
2               2  1iJBSr7s7jYXzM8EGcbK5b        Ingrid Michaelson;ZAYN   
3               3  6lfxq3CG4xtTiEg7opyCyx                  Kina Grannis   
4               4  5vjLSffimiIP26QG5WcN2K              Chord Overstreet   
...           ...                     ...                           ...   
23994       23994  4uPSL0Rx3JzbwigbJVndQ8                   Alex Schulz   
23995       23995  6GM93QLita5nZknl6G2s4M                 Noisia;Mefjus   
23996       23996  4iaAPRycdaqdtKukfa34ve    Viva La Panda;Next to Neon   
23997       23997  7d5rQgdrOfdcODLkyuwOrc                       Crazy P   
23998       23998  6xR2ZU8JUPrSZIKspHYH7A  Brazo Wa Afrika;Chymamusique   

                                              album_name  \
0        

In [ ]:
#@title Data Processing
# Memilih fitur numerik yang relevan
features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo']

# Inisialisasi scaler
scaler = StandardScaler()

# Mengambil subset data 23999 baris pertama
df_subset = df.head(23999)

# Menstandarkan fitur pada subset data
df_scaled = scaler.fit_transform(df_subset[features])

# Menghitung Cosine Similarity antar lagu pada subset
cosine_sim = cosine_similarity(df_scaled, df_scaled)
print("Cosine Similarity computed!")



Cosine Similarity computed!


Penjelasan:

* Memilih fitur numerik yang relevan untuk model content-based filtering. Fitur-fitur ini akan digunakan untuk menghitung kesamaan antar lagu.
* Menstandarkan data menggunakan StandardScaler untuk memastikan bahwa setiap fitur memiliki skala yang sama (misalnya, tidak ada fitur yang lebih dominan karena skala yang berbeda).
* Menggunakan cosine_similarity untuk menghitung kesamaan antar lagu berdasarkan fitur-fitur yang telah distandarkan.

In [ ]:
# Fungsi rekomendasi lagu
def get_recommendations(track_name, df_subset, cosine_sim):
    track_name_lower = track_name.lower()
    track_names_lower = df_subset['track_name'].str.lower()

    # Memeriksa apakah track_name ada dalam dataset
    if track_name_lower not in track_names_lower.values:
        print(f"Error: '{track_name}' not found in the dataset.")
        return [], []

    # Mencari index lagu yang sesuai
    idx = df_subset[track_names_lower == track_name_lower].index[0]

    # Mendapatkan skor kesamaan dengan semua track lainnya
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan skor kesamaan dari yang tertinggi ke yang terendah
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mengambil 10 rekomendasi teratas
    sim_scores = sim_scores[1:11]

    # Mengambil indeks lagu yang direkomendasikan
    track_indices = [i[0] for i in sim_scores]

    # Mengembalikan nama lagu dan skor kesamaannya
    return df_subset['track_name'].iloc[track_indices], [i[1] for i in sim_scores]

# Meminta input pengguna untuk nama lagu
track_name_input = input("Masukkan nama lagu yang ingin dicari rekomendasinya: ")

# Mengambil subset 23999 baris pertama untuk data
df_subset = df.head(23999)

# Menstandarkan fitur pada subset data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_subset[features])

# Menghitung Cosine Similarity antar lagu pada subset
cosine_sim = cosine_similarity(df_scaled, df_scaled)

# Mencari rekomendasi untuk lagu yang dimasukkan oleh pengguna
recommended_tracks, similarity_scores = get_recommendations(track_name_input, df_subset, cosine_sim)

# Menampilkan hasil rekomendasi
if len(recommended_tracks) > 0:  # Menggunakan len() untuk memeriksa apakah ada rekomendasi
    print(f"Recommended Tracks for '{track_name_input}':")
    for track, score in zip(recommended_tracks, similarity_scores):
        print(f"{track} (similarity: {score:.4f})")
else:
    print(f"Tidak ada rekomendasi untuk lagu '{track_name_input}' karena lagu tersebut tidak ditemukan.")


Masukkan nama lagu yang ingin dicari rekomendasinya: bad liar
Recommended Tracks for 'bad liar':
Bad Liar (similarity: 1.0000)
ugly (similarity: 0.9842)
Shallow (similarity: 0.9746)
Forever Young (Glee Cast Version) (similarity: 0.9720)
Em Teu Altar (similarity: 0.9652)
Million Years Ago (similarity: 0.9646)
我的歌 (similarity: 0.9606)
Closure (similarity: 0.9594)
Dear My Friend, (similarity: 0.9587)
Thursday (similarity: 0.9587)


* Fungsi ini digunakan untuk memberikan rekomendasi lagu berdasarkan Cosine Similarity.
* Pertama, kode memeriksa apakah nama lagu ada dalam dataset.
* Jika ada, fungsi akan menghitung kesamaan antara lagu yang diminta dengan lagu-lagu lainnya, mengurutkan hasilnya, dan mengembalikan 10 lagu teratas yang paling mirip.


In [ ]:
#@title Model 2
# Filter data untuk ukuran lebih kecil
df_subset = df.head(23999)  # Ambil subset data

# Membuat matriks interaksi
def create_interaction_matrix(df):
    """
    Membuat matriks interaksi pengguna-lagu berdasarkan 'popularity' (rating).
    """
    user_item_matrix = df.pivot_table(index='track_name', columns='artists', values='popularity', aggfunc='mean')
    return user_item_matrix.fillna(0)

# Membuat matriks interaksi dari subset data
user_item_matrix = create_interaction_matrix(df_subset)

# Normalisasi Data untuk Skala yang Konsisten
scaler = MinMaxScaler()
user_item_matrix_scaled = scaler.fit_transform(user_item_matrix.values)

# Convert User-Item Matrix ke Sparse Matrix setelah normalisasi
sparse_matrix_scaled = scipy.sparse.csr_matrix(user_item_matrix_scaled)

# Menyimpan track_names untuk referensi
track_names = user_item_matrix.index

* Mengambil subset data dari 23999 baris pertama memang berguna untuk mengurangi waktu komputasi

* Code ini menggunakan rating berdasarkan popularitas untuk menggambarkan hubungan antara track_name dan artists.

* Penggunaan Min Max Scaller untuk normalisasi data

* Membuat User-Item Interaction Matrix dengan menggunakan fitur popularity sebagai rating untuk collaborative filtering.
* Jika ada missing value (NaN), fungsi ini menggantinya dengan 0 (artinya lagu tersebut tidak dinilai oleh pengguna tertentu).

Menyiapkan Model SVD
* SVD (Singular Value Decomposition) akan digunakan untuk memecah matriks interaksi ke dalam faktor-faktor yang lebih rendah dan menemukan pola dalam data.

In [ ]:
# Model SVD Truncated untuk rekomendasi
def svd_recommendation_truncated(user_item_matrix, sparse_matrix, track_name, n_recommendations=10, n_components=1000):
    """
    Memberikan rekomendasi berdasarkan SVD untuk collaborative filtering.
    """
    # Melakukan Truncated SVD
    svd = TruncatedSVD(n_components=n_components, algorithm='randomized', random_state=42)
    reduced_matrix = svd.fit_transform(sparse_matrix)

    # Mendapatkan indeks lagu
    track_idx = user_item_matrix.index.get_loc(track_name)

    # Mendapatkan kesamaan antar lagu
    similar_tracks = reduced_matrix[track_idx]

    # Mendapatkan 10 lagu yang paling mirip
    similar_track_indices = similar_tracks.argsort()[-n_recommendations:][::-1]
    similar_track_names = track_names[similar_track_indices]

    return similar_track_names, similar_tracks[similar_track_indices]

* Fungsi ini memberikan rekomendasi berdasarkan model SVD (Singular Value Decomposition) untuk collaborative filtering.
* SVD mengurangi dimensi matriks interaksi pengguna-lagu untuk menemukan pola tersembunyi dan memberikan rekomendasi berdasarkan kesamaan antar lagu.

Pencarian Rekomendasi

* Fungsi untuk menampilkan pencarian lagu dan hasil dari SVD terkait popularity

In [ ]:
# Input lagu untuk rekomendasi
track_name_input = input("Masukkan nama lagu yang ingin dicari rekomendasinya: ").strip().lower()

# Memastikan lagu yang dicari ada dalam dataset
matching_tracks = [track for track in track_names if track_name_input in track.lower()]

if not matching_tracks:
    print(f"Lagu '{track_name_input}' tidak ditemukan dalam dataset.")
else:
    # Jika ada kecocokan, ambil lagu pertama sebagai referensi
    track_name_input = matching_tracks[0]

    # Mendapatkan rekomendasi
    recommended_tracks, similarity_scores = svd_recommendation_truncated(user_item_matrix, sparse_matrix_scaled, track_name_input)

    # Menampilkan rekomendasi
    print(f"Rekomendasi Lagu berdasarkan SVD untuk '{track_name_input}':")
    for track, score in zip(recommended_tracks, similarity_scores):
        print(f"{track} (similarity: {score:.4f})")

Masukkan nama lagu yang ingin dicari rekomendasinya: bad liar
Rekomendasi Lagu berdasarkan SVD untuk 'Bad Liar':
Affi Smart (similarity: 0.7259)
Aerials (similarity: 0.2309)
After Midnight (similarity: 0.1938)
6 Variations in F, K.54: 7. Variation VI (similarity: 0.1876)
Afterglow (similarity: 0.1389)
Affection (similarity: 0.1174)
24 Preludes, Op. 34: No. 14 in E-Flat Minor (Arr. L. Stokowski for Orchestra) [Live] (similarity: 0.1041)
Afterlife (similarity: 0.1002)
10 Variations in G, K.455 on "Unser dummer Pöbel meint" by C.W. Gluck: 3. Variation II (similarity: 0.0584)
Adiós Batata - Cerrero Dub Mix (similarity: 0.0417)


* Pada cell ini melakukan tahapan terkait Input lagu dari user, Mencari lagu ke dalam dataset, konfirmasi lagu, jika ada lagu cocok maka mengambil referensi lagu tersebut, mendapatkan hasil rekomendasi menggunakan SVD, Menampilkan top-N rekomendasi

In [ ]:
#@title Evaluasi

# Fungsi evaluasi precision
def evaluate_precision(track_name, df_subset, cosine_sim, top_n=10, threshold=0.8):
    # Mendapatkan rekomendasi untuk lagu input
    recommended_tracks, similarity_scores = get_recommendations(track_name, df_subset, cosine_sim)

    # Menyaring rekomendasi yang memiliki similarity > threshold
    relevant_recommendations = [score for score in similarity_scores if score >= threshold]

    # Hitung Precision: jumlah rekomendasi relevan / total rekomendasi
    precision = len(relevant_recommendations) / top_n if len(recommended_tracks) > 0 else 0

    # Menampilkan hasil evaluasi
    print(f"\nPrecision untuk lagu '{track_name}' : {precision:.4f}")
    return precision

# Evaluasi Precision untuk lagu yang dimasukkan oleh pengguna
precision = evaluate_precision(track_name_input, df_subset, cosine_sim)

# Jika ingin menghitung Precision untuk beberapa lagu, bisa lakukan loop di bawah ini:
track_names_input = ['ugly', 'Shallow', 'Forever Young (Glee Cast Version)', 'Em Teu Altar', 'Million Years Ago', '我的歌', 'Closure', 'Dear My Friend,', 'Thursday' ]  # List lagu yang ingin dievaluasi
for track in track_names_input:
    evaluate_precision(track, df_subset, cosine_sim)



Precision untuk lagu 'Bad Liar' : 1.0000

Precision untuk lagu 'ugly' : 1.0000

Precision untuk lagu 'Shallow' : 1.0000

Precision untuk lagu 'Forever Young (Glee Cast Version)' : 1.0000

Precision untuk lagu 'Em Teu Altar' : 1.0000

Precision untuk lagu 'Million Years Ago' : 1.0000

Precision untuk lagu '我的歌' : 1.0000

Precision untuk lagu 'Closure' : 1.0000

Precision untuk lagu 'Dear My Friend,' : 1.0000

Precision untuk lagu 'Thursday' : 1.0000


* Tujuan: Fungsi ini digunakan untuk menghitung precision berdasarkan rekomendasi lagu yang diberikan oleh model, yang dihitung menggunakan cosine similarity.
* Rumus Precision : Jumlah Rekomendasi relevan / Jumlah rekomendasi teratas (top_n)
* Parameter:
 1. track_name: Nama lagu yang ingin dicari rekomendasinya.
 2. df_subset: Data subset dari dataset yang akan digunakan dalam proses rekomendasi.
 3. cosine_sim: Matriks cosine similarity yang dihitung sebelumnya antara lagu-lagu dalam dataset.
 4. top_n: Jumlah rekomendasi teratas yang ingin dievaluasi (default 10).
 5. threshold: Batasan nilai similarity minimum agar suatu rekomendasi dianggap relevan (default 0.8).

Interpretasi Hasil

* Precision = 1.0000 berarti semua rekomendasi yang diberikan untuk lagu tersebut adalah relevan (nilai similarity antara lagu yang direkomendasikan dan lagu yang dimaksud sangat tinggi, melebihi nilai threshold yang telah ditentukan).
* Hasil ini menunjukkan bahwa model sangat baik dalam memberikan rekomendasi yang sesuai dan relevan untuk setiap lagu yang diuji.
Dalam konteks ini, jika Anda melihat hasil evaluasi dengan nilai Precision 1.0000 untuk lagu-lagu yang disebutkan (seperti 'Bad Liar', 'ugly', 'Shallow', dll.), itu berarti bahwa model memberikan rekomendasi yang tepat dan sesuai dengan harapan, dan tidak ada rekomendasi yang gagal memenuhi kriteria relevansi (threshold similarity >= 0.8).


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Data ground truth pada lagu
ground_truth = np.array(user_item_matrix)

# Menghitung MAE atau RMSE
mae = mean_absolute_error(ground_truth, user_item_matrix_scaled)
rmse = np.sqrt(mean_squared_error(ground_truth, user_item_matrix_scaled))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


Mean Absolute Error (MAE): 0.004457191773606404
Root Mean Squared Error (RMSE): 0.4485904294508946


* MAE mengukur kesalahan rata-rata tanpa mempertimbangkan apakah kesalahan tersebut besar atau kecil.
* RMSE memberikan lebih banyak bobot pada kesalahan besar dan lebih sensitif terhadap nilai-nilai ekstrem, jadi jika model membuat kesalahan besar dalam beberapa prediksi, RMSE akan lebih tinggi daripada MAE.

Interpretasi Hasil:
* MAE (0.0044):

MAE yang sangat kecil (dekat dengan 0) menunjukkan bahwa rata-rata kesalahan model dalam memprediksi rating/interaction cukup kecil. Artinya, model secara keseluruhan cukup akurat dalam memperkirakan nilai rating atau popularitas lagu.
Kesimpulan: Model ini cukup akurat, karena nilai MAE yang rendah mengindikasikan bahwa model tidak membuat banyak kesalahan.
 * RMSE (0.4486):

RMSE yang lebih besar menunjukkan bahwa meskipun MAE sangat kecil, ada beberapa kesalahan yang lebih besar yang memengaruhi hasil keseluruhan. Ini berarti ada beberapa prediksi yang cukup jauh dari nilai aktual.
RMSE cenderung memberikan penalti lebih besar untuk kesalahan yang lebih besar, jadi nilai RMSE ini menunjukkan bahwa meskipun sebagian besar prediksi baik, ada beberapa yang jauh lebih buruk.
Kesimpulan: Model memiliki beberapa prediksi yang sangat jauh dari nilai sebenarnya, meskipun rata-rata kesalahan tetap relatif kecil. Ini menunjukkan bahwa model cenderung lebih sensitif terhadap kesalahan besar.